```{contents}
```

## **Memory Store in LangGraph**

In LangGraph, a **Memory Store** is the infrastructure that enables **long-term persistence of state and knowledge across graph executions**, transforming LLM workflows from stateless pipelines into **continuous, evolving intelligent systems**.

It provides **durable context**, supports **multi-session reasoning**, and enables **agent learning, recovery, and personalization**.

---

### **1. Why Memory Store Exists**

Without memory, an LLM system behaves like:

> *“Solve the task, forget everything.”*

With memory, the system becomes:

> *“Learn, remember, adapt, improve.”*

| Capability             | Without Memory | With Memory |
| ---------------------- | -------------- | ----------- |
| User personalization   | ❌              | ✅           |
| Long conversations     | ❌              | ✅           |
| Autonomous agents      | ❌              | ✅           |
| Recovery after failure | ❌              | ✅           |
| Incremental learning   | ❌              | ✅           |

---

### **2. Types of Memory in LangGraph**

| Memory Type           | Purpose                      |
| --------------------- | ---------------------------- |
| **Short-Term Memory** | Current graph state          |
| **Thread Memory**     | Conversation/session context |
| **Checkpoint Memory** | Execution recovery           |
| **Long-Term Memory**  | Durable knowledge store      |
| **Agent Memory**      | Agent-specific experience    |
| **Vector Memory**     | Semantic knowledge retrieval |

---

### **3. Architecture of Memory Store**

```
LangGraph Runtime
     |
Shared State
     |
Checkpoint Store  ──┐
Thread Store       ├─→  Memory Store Layer
Long-Term Store    │
Vector Store  ─────┘
```

Memory is **externalized** from execution so that workflows can:

* pause
* crash
* migrate
* resume
* evolve

without losing intelligence.

---

### **4. Core Components**

| Component            | Function                       |
| -------------------- | ------------------------------ |
| **State Store**      | Persists current state         |
| **Checkpoint Store** | Saves execution snapshots      |
| **Thread Store**     | Maintains conversation history |
| **Vector Store**     | Semantic knowledge             |
| **Metadata Store**   | Audit, logs, versioning        |

---

### **5. Creating a Memory Store**

```python
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string("memory.db")
```

Attach memory to the graph:

```python
graph = builder.compile(checkpointer=memory)
```

Now every execution is **persisted automatically**.

---

### **6. Thread-Based Memory**

Each execution thread has isolated memory:

```python
config = {"configurable": {"thread_id": "user-123"}}
graph.invoke(input, config)
```

This enables:

* multi-user systems
* personalized assistants
* long conversations

---

### **7. Checkpointing & Recovery**

If execution crashes:

```python
graph.invoke(input, config)
```

LangGraph resumes from the **last checkpoint**.

| Feature        | Benefit          |
| -------------- | ---------------- |
| Crash recovery | No lost progress |
| Replay         | Debugging        |
| Rollback       | Error correction |
| Auditing       | Compliance       |

---

### **8. Long-Term Knowledge Memory (Vector Store)**

Used for **semantic recall**:

```python
from langchain.vectorstores import FAISS
vector_store = FAISS.from_texts(docs, embedding)
```

Integrated into nodes for retrieval-augmented reasoning.

---

### **9. Memory in Autonomous Agents**

Agents read and write memory continuously:

```
Perception → Reasoning → Action → Experience → Memory Update → Next Cycle
```

This enables:

* learning from mistakes
* preference adaptation
* behavior optimization

---

### **10. Production-Grade Memory Practices**

| Concern       | Solution         |
| ------------- | ---------------- |
| Scalability   | Redis / Postgres |
| Durability    | Cloud DB         |
| Privacy       | Encryption       |
| Governance    | Access control   |
| Cost          | TTL & pruning    |
| Observability | Audit logs       |

---

### **11. Mental Model**

> **LangGraph Memory = Brain of the system**
> State = Working memory
> Checkpoints = Episodic memory
> Vector store = Semantic memory

---

### **12. Minimal Example: Memory in Action**

```python
result1 = graph.invoke({"input": "Hello"}, config)
result2 = graph.invoke({"input": "What did I say?"}, config)
```

The system remembers because **memory persists across runs**.

---

### **Conclusion**

Memory Store elevates LangGraph from a **workflow engine** into a **persistent intelligent system** capable of learning, recovering, and evolving over time.


### Demonstration

In [3]:
from typing import TypedDict, List

class State(TypedDict):
    messages: List[str]


In [1]:
from langgraph.checkpoint.memory import InMemorySaver

memory = InMemorySaver()


In [4]:
from langgraph.graph import StateGraph, END

def chat_node(state: State):
    history = state.get("messages", [])
    user_msg = history[-1]
    
    response = f"I remember you said: {', '.join(history[:-1])}"
    return {"messages": history + [response]}

builder = StateGraph(State)
builder.add_node("chat", chat_node)
builder.set_entry_point("chat")
builder.add_edge("chat", END)

graph = builder.compile(checkpointer=memory)


In [6]:
config = {"configurable": {"thread_id": "user-1"}}
graph.invoke({"messages": ["Hello"]}, config)


{'messages': ['Hello', 'I remember you said: ']}

In [7]:
graph.invoke({"messages": ["Hello", "My name is Sanjeev"]}, config)


{'messages': ['Hello', 'My name is Sanjeev', 'I remember you said: Hello']}

In [9]:
graph.invoke({"messages": ["Hello", "My name is Sanjeev", "What is my name?"]}, config)


{'messages': ['Hello',
  'My name is Sanjeev',
  'What is my name?',
  'I remember you said: Hello, My name is Sanjeev']}